### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#Calculate the total number of schools
total_school_count = len(school_data)

#Calculate the total number of students
total_student_count = len(student_data)

#Calculate the total budget
total_budget = school_data['budget'].sum()

#Calculate the average math score
avg_math_score = student_data['math_score'].mean()

#Calculate the average reading score
avg_reading_score = student_data['reading_score'].mean()

#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_passing_rate = (avg_math_score+avg_reading_score)/2

#Calculate the percentage of students with a passing math score (70 or greater)
passing_math_rate = (student_data.loc[student_data['math_score'] >= 70, "grade"].count() /  total_student_count )*100

#Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_rate = (student_data.loc[student_data['reading_score'] >= 70, "grade"].count() /  total_student_count )*100

#Create a dataframe to hold the above results
summary_display_df = pd.DataFrame(
    {
    "Total Schools" : [total_school_count],
    "Total Students": [total_student_count],
    "Total Budget": [f"${total_budget:.2f}"],
    "Average Math Score" : [avg_math_score],
    "Average Reading Score ": [avg_reading_score],
    "% Passing Math" : [passing_math_rate],
    "% Passing Reading": [passing_reading_rate],
    "% Overall Passing Rate" : [overall_passing_rate]
    }
)

#Optional: give the displayed data cleaner formatting
summary_display_df.round(6)
summary_display_df.head()


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,$24649428.00,78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [166]:

school_data_summary = school_data_complete.groupby(['school_name'], as_index=True).agg(
    {
        'type' : 'first',
        'size' : 'first',
        'budget' : 'first',
        'math_score': 'mean',
        'reading_score': 'mean'
    }
)      


school_data_summary["Per Student Budget"] = (school_data_summary["budget"] / school_data_summary["size"]).map("${0:,.2f}".format)
school_data_summary["% Passing Math"]  =  school_data_complete.loc[school_data_complete["math_score"] >= 70,["reading_score", "school_name","Student ID"]].groupby('school_name')['Student ID'].count() / school_data_summary["size"] * 100
school_data_summary["% Passing Reading"]  =  school_data_complete.loc[school_data_complete["reading_score"] >= 70,["reading_score", "school_name","Student ID"]].groupby('school_name')['Student ID'].count() / school_data_summary["size"] * 100
school_data_summary["% Overall Passing Rate"]  = (school_data_summary["% Passing Math"] + school_data_summary["% Passing Reading"])  / 2

#cleaning
#Drop index label
del school_data_summary.index.name
#Format currency
school_data_summary["budget"] = school_data_summary["budget"].map("${0:,.2f}".format)
#Rename columns
school_data_summary = school_data_summary.rename(columns={'school_name' : "School Name",'type' : "School Type", 'math_score':'Average Math Score', 'reading_score': 'Average Reading Score' ,'size' : "Total Students", 'budget' : "Total School Budget"})
#Re-arrange columns
school_data_summary = school_data_summary[["School Type", "Total Students", "Total School Budget","Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate" ]]

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [167]:
school_data_summary.sort_values(by=["% Overall Passing Rate"], ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [41]:
school_data_summary.sort_values(by=["% Overall Passing Rate"]).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [171]:
score_by='math_score'

school_math_by_grade = school_data_complete.groupby(['school_name','grade'])
school_math_by_grade = pd.DataFrame({
    "School Name" : school_math_by_grade['school_name'].first(),
    "Grade" : school_math_by_grade['grade'].first(),
    "Score" : school_math_by_grade[score_by].mean()
})

school_math_by_grade_df = pd.DataFrame({
    'School Name' : school_math_by_grade['School Name'],
    '9th' : school_math_by_grade[school_math_by_grade['Grade'] == '9th']['Score'],
    '10th' : school_math_by_grade[school_math_by_grade['Grade'] == '10th']['Score'],
    '11th' : school_math_by_grade[school_math_by_grade['Grade'] == '11th']['Score'],
    '12th' : school_math_by_grade[school_math_by_grade['Grade'] == '12th']['Score']
})

school_math_by_grade_df = school_math_by_grade_df.groupby(['School Name']).agg({
        '9th' : 'sum', '10th' : 'sum', '11th' : 'sum', '12th' : 'sum'
})

#Format data
del school_math_by_grade_df.index.name
school_math_by_grade_df_styled = school_math_by_grade_df.style.set_properties(**{'text-align': 'left'})
school_math_by_grade_df_styled.format("{:,.6f}")
school_math_by_grade_df_styled.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

school_math_by_grade_df_styled

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [172]:
score_by='reading_score'

school_math_by_grade = school_data_complete.groupby(['school_name','grade'])
school_math_by_grade = pd.DataFrame({
    "School Name" : school_math_by_grade['school_name'].first(),
    "Grade" : school_math_by_grade['grade'].first(),
    "Score" : school_math_by_grade[score_by].mean()
})

school_math_by_grade_df = pd.DataFrame({
    'School Name' : school_math_by_grade['School Name'],
    '9th' : school_math_by_grade[school_math_by_grade['Grade'] == '9th']['Score'],
    '10th' : school_math_by_grade[school_math_by_grade['Grade'] == '10th']['Score'],
    '11th' : school_math_by_grade[school_math_by_grade['Grade'] == '11th']['Score'],
    '12th' : school_math_by_grade[school_math_by_grade['Grade'] == '12th']['Score']
})

school_math_by_grade_df = school_math_by_grade_df.groupby(['School Name']).agg({
        '9th' : 'sum', '10th' : 'sum', '11th' : 'sum', '12th' : 'sum'
})

#Format data
del school_math_by_grade_df.index.name
school_math_by_grade_df_styled = school_math_by_grade_df.style.set_properties(**{'text-align': 'left'})
school_math_by_grade_df_styled.format("{:,.6f}")
school_math_by_grade_df_styled.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

school_math_by_grade_df_styled

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
